In [1]:
import pandas as pd
import lightgbm as lgb
from lightgbm import LGBMClassifier as lgbc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score

In [277]:
# train = pd.read_pickle('/home/kai/data/shiyi/homecreditdata/fe_selected/train_0827.pkl')
# test = pd.read_pickle('/home/kai/data/shiyi/homecreditdata/fe_selected/test_0827.pkl')
train_bo2 = pd.read_pickle('/home/kai/data/shiyi/data/home_credit/train_bo2.pkl')
test_bo2 = pd.read_pickle('/home/kai/data/shiyi/data/home_credit/test_bo2.pkl')

test.SK_ID_CURR = test.SK_ID_CURR.astype(int)

In [239]:
train_pos = train[train['TARGET'] == 1]

train_neg = train[train['TARGET'] == 0].sample(frac=0.3)

train = pd.concat([train_pos, train_neg]).sample(frac=1)

In [276]:
lgb_params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'learning_rate': 0.01,
#         'max_bin': 511,
#         'min_data_in_leaf': 220,
        'num_leaves': 60, 
        'num_threads': 18, 
        'max_depth': 8,
        'min_child_weight': 49.45,
        'min_split_gain': 0.0552,
        'feature_fraction': 0.13,
        'feature_fraction_seed': 1992,
        'bagging_fraction': 0.966, 
        'bagging_seed': 1992,
        'scale_pos_weight': 1.7,
        'lambda_l1': 0.57, 
        'lambda_l2': 0.96,
        'metric': 'auc',
        'device': 'GPU'
    }

In [175]:
train_sel.shape

(153756, 2408)

In [268]:
# data_temp = data.drop(columns=['TARGET', 'SK_ID_CURR']+data_missing_cols.tolist())
def lgb_cv(X, y):
    print(X.shape)
    lgb_data = lgb.Dataset(X, y)

    eval_hist = lgb.cv(lgb_params, lgb_data, nfold=5, stratified=True, 
                       num_boost_round=50000, early_stopping_rounds=200, verbose_eval=300, seed=41)

    best_round = len(eval_hist['auc-mean'])
    print(best_round)
    print('lgb cv: ', eval_hist['auc-mean'][-1], ' std: ', eval_hist['auc-stdv'][-1])
    return best_round

In [284]:
def train_pred(X, y, X_test, best_rounds, subfilename='sub.csv'):
    print(X.shape)
    lgb_data = lgb.Dataset(X, y)
    model = lgb.train(lgb_params, lgb_data, num_boost_round=best_rounds)
    X_test_sub = X_test[['SK_ID_CURR']]
    X_test_sub['TARGET'] = model.predict(X_test)
    X_test_sub.to_csv(subfilename, index=False)
    return X_test_sub

In [ ]:
(153756, 2408)
211
lgb cv:  0.7847590475687548  std:  0.002821230328311794

In [ ]:
(153756, 2600)
207
lgb cv:  0.7832576715903944  std:  0.0019796626529290364

In [ ]:
220
lgb cv:  0.7821358235780864  std:  0.0027916802857769376

In [9]:
clf = lgbc(
    nthread=18,
    n_estimators=best_round,
    learning_rate=0.1,
    num_leaves=27,
    colsample_bytree=0.07,
    subsample=1,
    subsample_freq=0, ## disable subsampling
    max_depth=9,
    reg_alpha=1,
    reg_lambda=1,
    min_child_weight=84,
    min_child_samples=220,
    silent=-1,
    verbose=-1,
    device='CPU'
)

In [11]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=41)

In [12]:
%env JOBLIB_TEMP_FOLDER=/tmp

env: JOBLIB_TEMP_FOLDER=/tmp


In [168]:
scores = cross_val_score(clf, train_data.drop(columns=train_missing_cols), train['TARGET'], cv=skf,  scoring='roc_auc', verbose=100, n_jobs=-1)

Pickling array (shape=(2600,), dtype=object).
Memmaping (shape=(153756,), dtype=int64) to new file /tmp/joblib_memmaping_pool_2139_140258271359776/2139-140251827987456-4c6b22b1273e2bacc580c3efe0577e8a.pkl
Memmaping (shape=(2531, 153756), dtype=float64) to new file /tmp/joblib_memmaping_pool_2139_140258271359776/2139-140251827987456-190dd199414762f1d0c88ccf0eea0acc.pkl
[CV]  ................................................................
Memmaping (shape=(69, 153756), dtype=int64) to new file /tmp/joblib_memmaping_pool_2139_140258271359776/2139-140251827987456-04f9ca486e9c305d44836898968485ba.pkl
Pickling array (shape=(2531,), dtype=object).
Pickling array (shape=(69,), dtype=object).
Pickling array (shape=(2531,), dtype=int64).
Pickling array (shape=(69,), dtype=int64).
Memmaping (shape=(153756,), dtype=float64) to new file /tmp/joblib_memmaping_pool_2139_140258271359776/2139-140251827987456-f154161c8022fe344bf2614b6df2a5e2.pkl
Memmaping (shape=(153756,), dtype=int64) to old file /tmp

In [169]:
scores.mean(), scores.std()

(0.7836866006228165, 0.0033535790799512325)

In [154]:
scores.mean(), scores.std()

(0.7845090952399929, 0.0028690630448048983)

In [155]:
from sklearn.feature_selection import VarianceThreshold, chi2, SelectKBest, SelectPercentile, f_classif 

In [82]:
def variance_threshold_selector(data, threshold=0.5):
    selector = VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

In [127]:
train_data.shape

(153756, 2676)

In [160]:
train_sel.shape

(153756, 2408)

In [181]:
train_sel2_2 = train_sel1[selectKBest_selector(train_sel1.fillna(0), train['TARGET'], f_classif, 80)]#, 2000)

/home/kai/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [1516 1519 1520 1521 1522 1523 1536 1559 1564 1691 1692 1693 1694 1695
 1696 1697 1698 1699 1700 1701 1702 1703 1704 1705 1706 1707 1708 1709
 1710 2424 2425 2444 2445 2450 2451 2458 2459 2464 2465 2474 2475 2484
 2485 2494 2495 2504 2505 2516 2517 2520 2521 2548 2549 2576 2577] are constant.
  UserWarning)
/home/kai/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [139]:
train_data = train.drop(columns=['TARGET', 'SK_ID_CURR'])

In [25]:
from sklearn_pandas import DataFrameMapper

In [124]:
sample = train_data[['AMT_ANNUITY','AMT_ANNUITY_divide_AMT_GOODS_PRICE', 'AMT_ANNUITY_divide_AMT_REQ_CREDIT_BUREAU_DAY']].head().copy()

In [209]:
def sel_na(X, na_thresh):
    X_na = (X.isnull().sum() / len(X))#.sort_values(ascending = False)
    return X_na.index[X_na <= na_thresh]

In [194]:
from sklearn.feature_selection import SelectPercentile, f_classif 
def sel_percentile(X, y, score_func=f_classif, p=90):
    X = X.fillna(0)
    selector = SelectPercentile(score_func, p)
    selector.fit(X, y)
    return X.columns[selector.get_support(indices=True)]

In [233]:
def sel_corr_todrop(X, thresh, upper):
    if upper is None:
        corr_matrix = X.corr().abs()
        # Upper triangle of correlations
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
        pickle.dump(upper, open('upper.pkl', 'wb'))
    return [column for column in upper.columns if any(upper[column] > thresh)] 

In [278]:
X = train_bo2.drop(columns=['TARGET', 'SK_ID_CURR'])
y = train_bo2['TARGET']
X_test = test_bo2
bn = lgb_cv(X, y)
sub = train_pred(X, y, X_test, bn)


# X = X[sel_na(X, 0.95)]
# lgb_cv(X, y)

# to_drop = sel_corr_todrop(X, 0.95, upper)
# to_drop = [col for col in X.columns if col in to_drop]
# X = X.drop(columns=to_drop)
# X_test = X_test.drop(columns=to_drop)
# bn = lgb_cv(X, y)
# sub = train_pred(X, y, X_test, bn)

(307511, 2676)
[300]	cv_agg's auc: 0.777972 + 0.00375667
[600]	cv_agg's auc: 0.787999 + 0.00333269
[900]	cv_agg's auc: 0.792506 + 0.00302955
[1200]	cv_agg's auc: 0.794678 + 0.00291368
[1500]	cv_agg's auc: 0.795882 + 0.00291167
[1800]	cv_agg's auc: 0.796556 + 0.00290114
[2100]	cv_agg's auc: 0.796971 + 0.00284554
[2400]	cv_agg's auc: 0.797153 + 0.00283812
[2700]	cv_agg's auc: 0.797267 + 0.0028544
[3000]	cv_agg's auc: 0.797276 + 0.0028744
3041
lgb cv:  0.7973016864043533  std:  0.0028961028049985186
(307511, 2676)


/home/kai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [281]:
sub.SK_ID_CURR = sub.SK_ID_CURR.astype(int)

/home/kai/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [283]:
sub.to_csv('sub.csv', index=False)

In [ ]:
(307511, 407)
[300]	cv_agg's auc: 0.789638 + 0.00311958
[600]	cv_agg's auc: 0.794496 + 0.00306175
[900]	cv_agg's auc: 0.79592 + 0.00289928
[1200]	cv_agg's auc: 0.796327 + 0.00292147
[1500]	cv_agg's auc: 0.796412 + 0.00291771
[1800]	cv_agg's auc: 0.796403 + 0.00295967
1710
lgb cv:  0.7964685386921321  std:  0.002979441066024733
(307511, 407)

In [ ]:
(307511, 2676)
[300]	cv_agg's auc: 0.792076 + 0.0028967
385
lgb cv:  0.7923423900310747  std:  0.003006222150907127

In [ ]:
cols = sel_percentile(X, y)
X = X[cols]
X_test = X_test[cols]
bn = lgb_cv(X, y)
sub = train_pred(X, y, X_test, bn)

In [232]:
(54785, 2676)
249
lgb cv:  0.7794286226656331  std:  0.003618835037882991
(54785, 2639)
192
lgb cv:  0.7820200860381625  std:  0.0028846999986840774
(54785, 2375)
248
lgb cv:  0.7802523758999323  std:  0.0034383976850541867
(54785, 1245)
306
lgb cv:  0.7783458496867992  std:  0.0030065721961018066

,AMT_ANNUITY,AMT_ANNUITY_divide_AMT_GOODS_PRICE,AMT_ANNUITY_divide_AMT_REQ_CREDIT_BUREAU_DAY,AMT_ANNUITY_divide_AMT_REQ_CREDIT_BUREAU_HOUR,AMT_ANNUITY_divide_AMT_REQ_CREDIT_BUREAU_MON,AMT_ANNUITY_divide_AMT_REQ_CREDIT_BUREAU_QRT,AMT_ANNUITY_divide_AMT_REQ_CREDIT_BUREAU_WEEK,AMT_ANNUITY_divide_AMT_REQ_CREDIT_BUREAU_YEAR,AMT_ANNUITY_divide_CNT_CHILDREN,AMT_ANNUITY_divide_CNT_FAM_MEMBERS,...,unique_NAME_EDUCATION_TYPE_x,unique_NAME_EDUCATION_TYPE_y,unique_NAME_INCOME_TYPE_x,unique_NAME_INCOME_TYPE_y,unique_cnt_install_max,unique_cnt_install_mean,unique_cnt_install_min,unique_cnt_install_std,unique_cnt_install_sum,unique_prev
AMT_ANNUITY,NaN,0.08507,0.883824,0.97581,0.817785,0.850571,0.967757,0.639245,0.833132,0.706827,...,0.031639,0.023665,0.010817,0.071958,0.054729,0.065295,0.047737,0.029529,0.018280,0.001242
AMT_ANNUITY_divide_AMT_GOODS_PRICE,NaN,NaN,0.041619,0.08416,0.017544,0.068258,0.091411,0.082610,0.059282,0.046122,...,0.011920,0.024433,0.004819,0.035508,0.037370,0.033414,0.016637,0.021387,0.019977,0.011544


In [204]:
import pickle
upper = pickle.load(open('bo2_upper.pkl','rb'))
train_sel3 = train_data[sel_corr(train_data, 0.95, upper)]

In [79]:
data = pd.DataFrame({'pet':      ['cat', 'dog', 'dog', 'fish', 'cat', 'dog', 'cat', 'fish'], 
                     'children': [4., 6, 3, 3, 2, 3, 5, 4], 
                     'salary':   [90., 24, 44, 27, 32, 59, 36, 27]})

In [81]:
from sklearn.feature_selection import SelectKBest, chi2
mapper_fs = DataFrameMapper([(['children','salary'], SelectKBest(chi2, k=1))], df_out=True)
mapper_fs.fit_transform(data[['children','salary']], data['pet'])

,children_salary
0,90.0
1,24.0
2,44.0
3,27.0
4,32.0
5,59.0
6,36.0
7,27.0


In [ ]:
%%time
m = clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='auc', verbose=100)#, early_stopping_rounds=100)

In [48]:
from sklearn.model_selection import StratifiedKFold

In [49]:
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)

In [50]:
from sklearn.model_selection import cross_val_score

In [56]:
scores = cross_val_score(clf, train.drop(columns=['TARGET', 'SK_ID_CURR']), train['TARGET'], cv=folds, scoring='roc_auc')

array([0.78733226, 0.79320251, 0.79428609, 0.79896442, 0.7967143 ])

In [60]:
np.array([0.78733226, 0.79320251, 0.79428609, 0.79896442, 0.7967143]).mean()

0.794099916